<a href="https://colab.research.google.com/github/minicks/BigI/blob/master/titanic5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import

In [0]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
import lightgbm

데이터 불러오기

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/test.csv')
combine = [train_df, test_df]

# 전처리

## 전처리('Embarked') 문자를 숫자값으로

In [0]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} )
    for index in dataset[dataset['Embarked'].isnull()].index:
        dataset.loc[index,'Embarked'] = 0



# 내가 한 거
# for index in train_df[train_df['Embarked'] == 'C'].index:
#   train_df.loc[index,'Embarked'] = 0

# for index in train_df[train_df['Embarked'] == 'Q'].index:
#   train_df.loc[index,'Embarked'] = 1

# for index in train_df[train_df['Embarked'] == 'S'].index:
#   train_df.loc[index,'Embarked'] = 2

# for index in train_df[train_df['Embarked'].isnull()].index:
#   train_df.loc[index,'Embarked'] = 0

## 전처리('Sex') 문자를 숫자값으로

In [0]:
for dataset in combine:
  dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} )

# 내가 한 거
# for index in train_df[train_df['Sex'] == 'male'].index:
#   train_df.loc[index,'Sex'] = 0

# for index in train_df[train_df['Sex'] == 'female'].index:
#   train_df.loc[index,'Sex'] = 1

## 나이 변환 후 카테고리나누기

In [0]:
for dataset in combine:
  for index in dataset[dataset['Age'].isnull()].index:
    if (dataset.loc[index,'SibSp']==0) & (dataset.loc[index,'Parch']==0):
      dataset.loc[index,'Age'] = 32.0
    elif (dataset.loc[index,'SibSp']==0) & ((2>=dataset.loc[index,'Parch']) & (dataset.loc[index,'Parch'] > 0)):
      dataset.loc[index,'Age'] = 26.0
    elif (dataset.loc[index,'SibSp'] == 0) & (dataset.loc[index,'Parch'] > 2):
      dataset.loc[index,'Age'] = 32.0
    elif ((2 >= dataset.loc[index,'SibSp']) & (dataset.loc[index,'SibSp'] > 0)) & (dataset.loc[index,'Parch']==0):
      dataset.loc[index,'Age'] = 32.0
    elif ((2 >= dataset.loc[index,'SibSp']) & (dataset.loc[index,'SibSp'] > 0)) & ((2>=dataset.loc[index,'Parch']) &(dataset.loc[index,'Parch'] > 0)):
      dataset.loc[index,'Age'] = 23.0
    elif ((2>=dataset.loc[index,'SibSp']) & (dataset.loc[index,'SibSp'] > 0)) & (dataset.loc[index,'Parch'] > 2):
      dataset.loc[index,'Age'] = 40.0
    elif (dataset.loc[index,'SibSp']>2) & (dataset.loc[index,'Parch']==0):
      dataset.loc[index,'Age'] = 31.0
    elif (dataset.loc[index,'SibSp']>2) & ((2>=dataset.loc[index,'Parch']) &(dataset.loc[index,'Parch'] > 0)):
      dataset.loc[index,'Age'] = 8.0
    elif (dataset.loc[index,'SibSp']>2) & (dataset.loc[index,'Parch'] > 2):
      dataset.loc[index,'Age'] = 32.0


In [0]:
# 분포 확인
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [0]:
train_df = train_df.drop(['AgeBand'], axis=1)

In [0]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16.336, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16.336) & (dataset['Age'] <= 32.252), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32.252) & (dataset['Age'] <= 48.168), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48.168) & (dataset['Age'] <= 64.084), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64.084, 'Age'] = 4


## Name을 Title로 변환

In [0]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)


## Fare을 카테고리

In [0]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 5)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [0]:
train_df = train_df.drop(['FareBand'], axis=1)

In [0]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.854, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.854) & (dataset['Fare'] <= 10.5), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 10.5) & (dataset['Fare'] <= 21.679), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 21.679) & (dataset['Fare'] <= 39.688), 'Fare']   = 3
    dataset.loc[ dataset['Fare'] > 39.688, 'Fare'] = 4

## 원핫인코딩

In [0]:
for dataset in combine:
  dataset["Embarked_C"] = dataset["Embarked"] == "C"
  dataset["Embarked_S"] = dataset["Embarked"] == "S"
  dataset["Embarked_Q"] = dataset["Embarked"] == "Q"

def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data


dummy_columns = ["Sex", "Pclass", "Embarked"]
train_df = dummy_data(train_df, dummy_columns)
test_df = dummy_data(test_df, dummy_columns)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## col 삭제

In [0]:
# Name,Ticket,Cabin col 삭제
df_1 = train_df.drop(columns=['Name','Ticket','Cabin'])
df_2 = test_df.drop(columns=['Name','Ticket','Cabin'])

In [0]:
#train,valid 나누기 
train0,test0 = train_test_split(df_1,test_size=0.3, random_state=1)

## None 값 채우기

In [0]:
train0=train0.fillna(0.0)
test0=test0.fillna(10.0)

In [0]:
df_2=df_2.fillna(20.0)

## 회귀 넣기 전 데이터 *처리*

In [0]:
Ytrain = train0["Survived"]
Xtrain = train0.drop("Survived",axis = 1)
Ytest = test0["Survived"]
Xtest = test0.drop("Survived",axis = 1)
Ydf_1 = df_1["Survived"]
Xdf_1 = df_1.drop("Survived",axis = 1)

In [0]:
Xtrain=Xtrain.astype('float32')
Ytrain=Ytrain.astype('float32')
Xtest=Xtest.astype('float32')
Ytest=Ytest.astype('float32')
Xdf_1=Xdf_1.astype('float32')
Ydf_1=Ydf_1.astype('float32')
df_2=df_2.astype('float32')


## 전처리 test

In [0]:
for index in train_df[train_df['Age'].isnull()].index:
  train_df=train_df.drop(index=index)

In [0]:
for index in test_df[test_df['Age'].isnull()].index:
  test_df=test_df.drop(index=index)

test

In [0]:
train_df[((2>=train_df['SibSp']) &(train_df['SibSp'] > 0)) & (train_df['Parch'] > 2)].describe()

In [0]:
train_df[((2>=train_df['SibSp']) &(train_df['SibSp'] > 0)) & ((2>=train_df['Parch']) &(train_df['Parch'] > 0))].describe()

In [0]:
train_df[((2>=train_df['SibSp']) &(train_df['SibSp'] > 0)) & (train_df['Parch'] == 0)].describe()

In [0]:
df_1.info()

In [0]:
df_2.info()

In [0]:
Xdf_1

In [0]:
Xtrain

In [0]:
for a in XXtrain.isnull():
  print(a)

# 회귀 분석

Randomforest

In [43]:
random_forest = RandomForestClassifier(n_estimators=600)
random_forest.fit(Xtrain, Ytrain)
Ftrain = random_forest.predict(Xtrain)
Ydf_2 = random_forest.predict(df_2)
random_forest.score(Xtest,Ytest )
random_forest.score(Xtrain, Ytrain )
random_forest.score(df_2, Ydf_2 )
acc_random_forest1 = round(random_forest.score(Xtrain, Ftrain ) * 100, 2)
acc_random_forest2 = round(random_forest.score(Xtest,Ytest ) * 100, 2)
acc_random_forest3 = round(random_forest.score(df_2,Ydf_2 ) * 100, 2)
print('train : ' , acc_random_forest1)
print('valid : ' , acc_random_forest2)
print('test : ' ,acc_random_forest3)

train :  100.0
valid :  76.12
test :  100.0


Logistic Regression

In [76]:
logreg = LogisticRegression()
logreg.fit(Xtrain, Ytrain)
Ydf_2 = logreg.predict(df_2)
acc_log = round(logreg.score(Xtrain, Ytrain) * 100, 2)
acc_log2 = round(logreg.score(Xtest,Ytest) * 100, 2)
print(acc_log)
print(acc_log2)

82.99
77.99


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Support Vector Machines

In [21]:
svc = SVC()
svc.fit(Xtrain, Ytrain)
Ydf_2 = svc.predict(df_2)
acc_svc = round(svc.score(Xtrain, Ytrain) * 100, 2)
acc_svc

63.56

KNN

In [22]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(Xtrain, Ytrain)
Ydf_2 = knn.predict(df_2)
acc_knn = round(knn.score(Xtrain, Ytrain) * 100, 2)
acc_knn

80.58

Gaussian Naive Bayes

In [23]:
gaussian = GaussianNB()
gaussian.fit(Xtrain, Ytrain)
Ydf_2 = gaussian.predict(df_2)
acc_gaussian = round(gaussian.score(Xtrain, Ytrain) * 100, 2)
acc_gaussian

81.54

Perceptron

In [24]:
perceptron = Perceptron()
perceptron.fit(Xtrain, Ytrain)
Ydf_2 = perceptron.predict(df_2)
acc_perceptron = round(perceptron.score(Xtrain, Ytrain) * 100, 2)
acc_perceptron

63.72

Linear SVC

In [25]:
linear_svc = LinearSVC()
linear_svc.fit(Xtrain, Ytrain)
Ydf_2 = linear_svc.predict(df_2)
acc_linear_svc = round(linear_svc.score(Xtrain, Ytrain) * 100, 2)
acc_linear_svc

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


77.69

Stochastic Gradient Descent

In [26]:
sgd = SGDClassifier()
sgd.fit(Xtrain, Ytrain)
Ydf_2 = sgd.predict(df_2)
acc_sgd = round(sgd.score(Xtrain, Ytrain) * 100, 2)
acc_sgd

64.53

Decision Tree

In [49]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(Xtrain, Ytrain)
Ydf_2 = decision_tree.predict(df_2)
acc_decision_tree = round(decision_tree.score(Xtrain, Ytrain) * 100, 2)
acc_decision_tree

100.0

그레디언트 부스트

In [74]:
model_grad = GradientBoostingClassifier(n_estimators=100, max_depth=2, random_state=0)
model_grad.fit(Xtrain, Ytrain)
Ydf_2 = model_grad.predict(df_2)
acc_model_grad = round(model_grad.score(Xtrain, Ytrain) * 100, 2)
acc_model_grad

89.41

XGBoost

In [60]:
model_xgb = xgboost.XGBClassifier(n_estimators=100, max_depth=1, random_state=0)
model_xgb.fit(Xtrain, Ytrain)
Ydf_2 = model_xgb.predict(df_2)
acc_model_xgb = round(model_xgb.score(Xtrain, Ytrain) * 100, 2)
acc_model_xgb

82.99

LightGBM

In [61]:
model_lgbm = lightgbm.LGBMClassifier(n_estimators=100, max_depth=1, random_state=0)
model_lgbm.fit(Xtrain, Ytrain)
Ydf_2 = model_lgbm.predict(df_2)
acc_model_lgbm = round(model_lgbm.score(Xtrain, Ytrain) * 100, 2)
acc_model_lgbm

82.34

분석

In [63]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree','GradientBoosting','XGBoost','LightGBM'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest1, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree, acc_model_grad , acc_model_xgb , acc_model_lgbm]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,100.00
8,Decision Tree,100.00
9,GradientBoosting,89.41
2,Logistic Regression,82.99
10,XGBoost,82.99
11,LightGBM,82.34
4,Naive Bayes,81.54
1,KNN,80.58
7,Linear SVC,77.69
6,Stochastic Gradient Decent,64.53


In [47]:
random_forest = RandomForestClassifier(n_estimators=600)
random_forest.fit(Xdf_1, Ydf_1)
Ftrain = random_forest.predict(Xtrain)
Ydf_2 = random_forest.predict(df_2)
random_forest.score(Xtest,Ytest )
random_forest.score(Xtrain, Ytrain )
random_forest.score(df_2, Ydf_2 )
acc_random_forest1 = round(random_forest.score(Xtrain, Ftrain ) * 100, 2)
acc_random_forest2 = round(random_forest.score(Xtest,Ytest ) * 100, 2)
acc_random_forest3 = round(random_forest.score(df_2,Ydf_2 ) * 100, 2)
print('train : ' , acc_random_forest1)
print('valid : ' , acc_random_forest2)
print('test : ' ,acc_random_forest3)

train :  100.0
valid :  100.0
test :  100.0


In [71]:
model_grad = GradientBoostingClassifier(n_estimators=100, max_depth=2, random_state=0)
model_grad.fit(Xdf_1, Ydf_1)
Ydf_2 = model_grad.predict(df_2)
acc_model_grad = round(model_grad.score(Xdf_1, Ydf_1) * 100, 2)
acc_model_grad2 = round(model_grad.score(df_2, Ydf_2) * 100, 2)
print(acc_model_grad)
print(acc_model_grad2)

85.3
100.0


In [72]:
logreg = LogisticRegression()
logreg.fit(Xdf_1, Ydf_1)
Ydf_2 = logreg.predict(df_2)
acc_log = round(logreg.score(Xdf_1, Ydf_1) * 100, 2)
acc_log2 = round(logreg.score(df_2, Ydf_2) * 100, 2)
print(acc_log)
print(acc_log2)

80.36
100.0


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# 파일 저장

In [0]:
submission = pd.DataFrame({
        "PassengerId": df_2["PassengerId"],
        "Survived": Ydf_2
    })
submission.to_csv('boost3'+'titanic_SUBMISSION.csv', index=False)

# 정보 모으기

In [0]:
print(train_df.sort_values(by='Survived'))
print(train_df.columns.values)
print('======================================================')
print(test_df)
print(test_df.columns.values)
print('======================================================')
print(combine)
print('======================================================')

In [0]:
print(train_df.min())
print(train_df.max())
train_df.describe(include='all')

In [0]:
train_df.info()

In [0]:
for a in train_df.index[train_df['Age'].isnull()]:
  print(a)

In [0]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [0]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Cabin', alpha=.5, bins=20)
grid.add_legend();

In [0]:
for index in train_df[train_df['Age'].isnull()].index:
  print(train_df.loc[index,['Pclass','SibSp','Parch']])

In [0]:

for index in train_df[(train_df['Age'].isnull()) & (train_df['SibSp'] == 0) & (train_df['Parch'] == 0)].index:
  print(train_df.loc[index,['Pclass','SibSp','Parch','Fare','Embarked','Cabin']])

In [0]:
train_df.corr(method='pearson')

In [0]:
train_df.info()

In [0]:
train_df.corr(method='pearson')

In [0]:
train_df.describe()

In [0]:
for index in train_df[train_df['Pclass'] == 1].index:
  train_df.loc[index,'Pclass'] = 'a'

In [0]:
for index in train_df[train_df['Pclass'] == 3].index:
  print(train_df.loc[index,'Cabin'])

# 새 섹션

In [0]:
Ytrain = train0["Survived"]
Xtrain = train0.drop("Survived",axis = 1)
Xtest = test0.drop("Survived",axis = 1)

In [0]:
Xtrain=Xtrain.astype('float32')
Ytrain=Ytrain.astype('float32')
Xtest=Xtest.astype('float32')

In [0]:
for index in Xtrain[Xtrain.isnull()].index:
  print(Xtrain.loc[index])

In [0]:
Xtrain[Xtrain.isnull()]

In [0]:
Xtest.info()

In [0]:
Xtrain[Xtrain.isnull()].index

In [0]:
Ypred

In [0]:
df_2